In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
config = {
    'domain': 'spouse',
    'debug': False,
    'lf_source': 'gradturk',
    'max_explanations': None,
    'apply_filters': False,
    'seed': 12345
}

In [3]:
# Get DB connection string and add to globals
# NOTE: $SNORKELDB must be set before any snorkel imports
import os

default_db_name = 'babble_' + config['domain'] + ('_debug' if config.get('debug', False) else '')
DB_NAME = config.get('db_name', default_db_name)
if 'postgres' in config and config['postgres']:
    DB_TYPE = 'postgres'
else:
    DB_TYPE = 'sqlite'
    DB_NAME += '.db'
DB_ADDR = "localhost:{0}".format(config['db_port']) if 'db_port' in config else ""
os.environ['SNORKELDB'] = '{0}://{1}/{2}'.format(DB_TYPE, DB_ADDR, DB_NAME)
print("$SNORKELDB = {0}".format(os.environ['SNORKELDB']))

$SNORKELDB = sqlite:///babble_spouse.db


In [4]:
from snorkel import SnorkelSession
session = SnorkelSession()

# Resolve config conflicts (nb_config > local_config > global_config)
from snorkel.contrib.babble.pipelines import merge_configs, get_local_pipeline
config = merge_configs(config)

from snorkel.models import candidate_subclass
candidate_class = candidate_subclass(config['candidate_name'], config['candidate_entities'])

pipeline = get_local_pipeline(config['domain'])
pipe = pipeline(session, candidate_class, config)

Overwriting lf_source=gradturk30 to lf_source=gradturk
Overwriting domain=None to domain=spouse
Overwriting print_freq=1 to print_freq=5
Overwriting LF_acc_prior_weight_default=1.0 to LF_acc_prior_weight_default=0.5
Overwriting decay=0.95 to decay=0.99
Overwriting init_class_prior=0 to init_class_prior=-1.15
Overwriting reg_param=0.1 to reg_param=0.5
Overwriting seed=0 to seed=12345
Overwriting babbler_candidate_split=1 to babbler_candidate_split=[0, 1, 2]
Overwriting disc_model_class=lstm to disc_model_class=logreg
Overwriting apply_filters=True to apply_filters=False


In [5]:
for split in [0,1,2]:
    print(session.query(pipe.candidate_class).filter(
        pipe.candidate_class.split == split).count())

22195
2796
2697


In [6]:
from snorkel.annotations import load_label_matrix
good_label_matrix = load_label_matrix(pipe.session, split=1)

In [7]:
good_label_matrix

<2796x0 sparse matrix of type '<type 'numpy.int64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [ ]:
num_lfs = good_label_matrix.shape[1]

In [ ]:
good_signatures = set([hash(good_label_matrix[:,i].nonzero()[0].tostring()) for i in range(num_lfs)])

In [ ]:
%time pipe.collect()

In [ ]:
%time pipe.label(split=1)

In [ ]:
all_label_matrix = load_label_matrix(session, split=1)
num_lfs = all_label_matrix.shape[1]
print(num_lfs)

In [ ]:
all_parses = pipe.babbler.get_parses()
print(len(all_parses))

In [ ]:
matching_parses = []

for i in range(num_lfs):
    signature = hash(label_matrix[:,i].nonzero()[0].tostring())
    if signature in signatures:
        print(signature)
        matching_parses.add

In [ ]:
all_signatures = [hash(label_matrix[:,i].nonzero()[0].tostring()) for i in range(num_lfs)]

In [ ]:
pipe.explanations